# Workshop 3: Let's get our model!

**🎯 Goals of this Workshop**
1. Understand the best way to split the data into train and test
2. Define the performance metrics that you are going to use in the evaluation of your model 
3. Develop a machine learning (ML) model to either:
  - predict $SaO_2$ values -> regression
  - predict the gap between $SaO_2$ and $SpO_2$ -> regression
  - detect cases of Hidden Hypoxemia (HH) -> classification

  The developed model can either be linear or non-linear.
4. Implement grid-search to further optimize parameters.
5. Assess what were the most relevant features for the regression/classification.


**✏️ Expected Deliverables**
 - Developed models with the performance metrics and feature importance properly reported.



**❗ Highlighted Pitfall(s)**
- Outcome leakage
- Suboptimal metrics for model evaluation
- No improvement compared to the presented baseline / Models not learning
- Overly complex models

## Table of Contents

1. Library imports
2. Implement an ML pipeline with grid-search parameter tunning
3. Model Evaluation (using meaningful metrics and assessing feature importance)

## 1. Setup Environment

You can add more libraries if you are familiar with them for your own model. But use these packages for the first part only.

In [ ]:
#!pip install shap

In [ ]:
#!pip install yellowbrick

In [ ]:
# Data reading in Dataframe format and data preprocessing
import pandas as pd
pd.set_option("display.max_columns", 160)
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset Creation
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV

# Dataset Processing
from sklearn import datasets, linear_model, metrics
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Model Development
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Ridge 
from sklearn.svm import SVC

# Model Evaluation
from sklearn.metrics import r2_score, confusion_matrix, ConfusionMatrixDisplay
from yellowbrick.classifier import ClassificationReport, ClassPredictionError
from yellowbrick.regressor import ResidualsPlot, PredictionError

# Feature Importance
import shap

# # For those who use Google Colab
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/"Colab Notebooks"/"your_path_to_drive_root_dir"

## 2. Split dataset into train and test

Before any data standardization, it is crucial to split the data into two groups, training and testing (so that there is no data leakage). To start, we will put 70% of our data into our training set, and 30% of our data into the testing set. Feel free to try other train/test splits, such as 75%/25%, or 80%/20%.

You might also want to ensure that minority groups are represented in both train and test sets. For that, you can use more specific train-test-split methods, read more about it [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split). Or split according to the time someone entered the hospital (important if policies change).

###### ✏️ Set your path to the dataset 

In [ ]:
# Dataset Path:
out_train = './train.csv'
out_test = './test.csv'

df_train = pd.read_csv(out_train)
df_test = pd.read_csv(out_test)

In [ ]:
df_train.head()

###### ✏️ Split your labels from the remaining dataset

In [ ]:
label_cols=['hidden_hypoxemia', 'SaO2']

y_train = df_train[label_cols]
X_train = df_train.drop(columns=label_cols)

y_test = df_test[label_cols]
X_test = df_test.drop(columns=label_cols)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

### Get label vectors for both possibilities: regression and classification

If you want to use regression or classification, the labels used to train the model will be different. In the case of regression, you will have a numerical variables with SaO2 values, whereas in classification you need to define classes for hidden hypoxemia.

In [ ]:
y_train_c = y_train[['hidden_hypoxemia']].values
y_test_c = y_test[['hidden_hypoxemia']].values
y_train_r = y_train[['SaO2']].values
y_test_r = y_test[['SaO2']].values

## 3. Implement ML Pipeline

### 3.1 Naive Model Implementation

What would our accuracy be if we predicted the most likely class? In this case, our prediction would simply be 1 or 0 for every patient. Using our training dataset we can create a naive model that predicts the most likely class for every patient.


In [ ]:
if np.sum(y_train_c == 0):
    y_preds_cc = [1 for _ in range(len(y_test_c))]
else:
    y_preds_cc = [0 for _ in range(len(y_test_c))]

# Check accuracy score
accuracy_score = np.mean(y_preds_cc == y_test_c)
print("Accuracy: {:.4f}".format(accuracy_score))

###### ✏️ Test a naive approach to have a performance baseline

In [ ]:
# Code here !

...

### 3.2 Data Normalization

Many machine learning models are directly influenced by the scale of the features that you input to the model. Therefore, it is important to normalize the scale of values used in your pipeline.

One of the most common methods used is data standardization, but the decision should be taken considering the specific use case or model that you are developing. If you are curious, you can further read about this topic [here](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py).

It is important to consider that in a dataset one might have several types of variables: categorical, continuous, binary. For both categorical and binary variables, it is not a good practice to apply standardization methods. Therefore, it is important to distinguish the different type of variables that one might have and apply normalization methods only to those that make sense.

In [ ]:
feats = df_train.columns
for feat in feats:
    print(feat)
    print(df_train[feat].value_counts())

In [ ]:
binary_variables = ['invasive_vent', 'language', 'gender', 'rrt', 'mortality_in', 'HFNC', 
                    'InvasiveVent', 'NonInvasiveVent', 'None_ventilation', 'SupplementalOxygen',  'Tracheostomy', 
                    'Asian', 'Black', 'Hispanic', 'Other_race_group', 'White', 
                    'Medicaid', 'Medicare', 'Other']

In [ ]:
X_train_continuous = X_train.drop(columns=binary_variables)
X_test_continuous = X_test.drop(columns=binary_variables)
X_train_binary = X_train[binary_variables]
X_test_binary = X_test[binary_variables]

In [ ]:
preprocessor = make_pipeline(StandardScaler())

preprocessor

In [ ]:
train_preprocessed = preprocessor.fit_transform(X_train_continuous)
pd.DataFrame(train_preprocessed, columns=preprocessor.get_feature_names_out())

In [ ]:
test_preprocessed = preprocessor.transform(X_test_continuous)

Join the processed continuous variables with the binary ones to then feed to the model.

In [ ]:
X_train_processed = pd.concat([X_train_binary, pd.DataFrame(train_preprocessed, columns=preprocessor.get_feature_names_out())], axis=1, join='inner')
X_test_processed = pd.concat([X_test_binary, pd.DataFrame(test_preprocessed, columns=preprocessor.get_feature_names_out())], axis=1, join='inner')

Note: After transforming your data, it is important to further explore it's distribution and ensure that the transformations applied make sense and resulted on what was expected.

###### ✏️ Implement your own data normalization strategy

In [ ]:
# (Optional)

# Check your normalization, e.g. summary statistics, histograms, scatter plots, etc.

### 3.3 Linear Regression Baseline

On of the easiest Machine Learning model is **linear regression**. It assumes that the target variable can be written as a linear combination of the features.

In many problems, this solution can be a very strong baseline. Thanks to its simplicity, it can be fitted very quickly even when the number of features is high. 

In general, wrapping together the features, preprocessing and fitting the estimator in a single pipeline makes it easier to transform, fit and predict the data.

In [ ]:
lmodel = linear_model.LinearRegression()

lpipeline = make_pipeline(lmodel)
lpipeline

We train the model by calling the `fit` function on the train data.

In [ ]:
# Train the model using the training set
lpipeline.fit(X_train_processed.values, y_train_r)

**Interpretation**: Because the linear model gives one weight to each feature, we can easily explore how it modeled our target covariate by visualizing the coefficients.   


In [ ]:
print('Examine regression coefficients:')
linear_coefficients = pd.DataFrame(
    lpipeline.named_steps.linearregression.coef_, columns=lpipeline[:-1].get_feature_names_out()
)

linear_coefficients

✏️ Don't you see anything strange with some of these coefficients ? 

These weird coefficients are caused by multiple features measuring almost the same thing --a phenoma called multi-colinearity. Giving redundant information to a linear model, makes it predict unprecise and noisy coefficients.  
You can learn more about [the limitations of the linear model here](https://inria.github.io/scikit-learn-mooc/python_scripts/linear_models_regularization.html).

A simple solution is to force the model to avoid extreme coefficients, by adding a *regularization*. The subsequent model is called a Ridge regression.

In [ ]:
# Code here !

# Train the model using the training set
ridge_model = ...
ridge_pipeline = ...
ridge_pipeline.fit(...)

Examine the regression coefficients for the Ridge estimator. It looks much more reasonable.


In [ ]:
ridge_coefficients = pd.DataFrame(
    {"coefficients": ridge_pipeline.named_steps.ridge.coef_[0], "feature_names": ridge_pipeline[:-1].get_feature_names_out()}
).set_index("feature_names").sort_values("coefficients", ascending=False).transpose()

print('Coefficients: ')
ridge_coefficients

We also can explore the errors of the model --called [residuals](https:/www.ncl.ac.uk/webtemplate/ask-assets/external/maths-resources/statistics/regression-and-correlation/residuals.html#:~:text=Definition,yi%E2%88%92%5Eyi). 

In [ ]:
# create residual error plot
plt.scatter(ridge_pipeline.predict(X_train_processed), ridge_pipeline.predict(X_train_processed) - y_train_r, color="green", s=10, label='Train data')
plt.scatter(ridge_pipeline.predict(X_test_processed), ridge_pipeline.predict(X_test_processed) - y_test_r, color="navy", s=10, label='Test data')
plt.hlines(y=0, xmin=0.4, xmax=1, linewidth=2, color="black", linestyle="dotted")
plt.legend(loc='upper left')
plt.title("Residual errors")
plt.xlabel(r"True SaO2 value: $y$")
plt.ylabel(r"Error on the prediction: $\hat y  - y$")
plt.show()

### 3.4 SVM Classification Baseline

For the classification task, you might choose from a wide range of models that you think are most suitable. Here you have the example of the implementation of a Support Vector Machine (SVM).

Before training the model, we need to set a given number of parameters - i.e. hyperparameters - which will be critical in building robust and accurate models. They help us find the balance between bias and variance and thus, prevent the model from overfitting or underfitting.Keep in mind that if you increase the range of hyperparameters to be tested, the training time will increase significantly. If you want more information read [here](https://towardsdatascience.com/hyperparameter-tuning-for-support-vector-machines-c-and-gamma-parameters-6a5097416167).

In [ ]:
# Code here !

# Defining parameter range
param_grid = {'C': [0.1, 100], 
              'gamma': [1, 0.0001],
              'kernel': ['rbf']} 

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)

# fitting the model for grid search
grid.fit(...)

# generate prediction for test dataset
cpredictions = grid.predict(...)

print('Score: {}'.format(grid.score(X_test_processed.values, y_test_c.ravel())))

As you can observe, the scoring is almost 100%! Great, right? Well, let's take a closer look at the confusion matrix for a deeper analysis:

In [ ]:
cm_display = ConfusionMatrixDisplay(confusion_matrix(y_test_c, cpredictions))
cm_display.plot()
plt.show()

It looks like we have a really imbalanced dataset, where we have only 110 measurements with hidden hypoxia compared to 6073 normal ones. What should we do then?

One can take several approaches when dealing with imbalanced datasets:
- removing the number of datapoints for the majority class to match the number on the minority class. However, this might lead you to loose a lot of information.
- upsample the minority class and generate synthetic data on it, using for example the SMOTE algorithm. This approach has the problem of maintaining the distribution of each variable for that class and might not provide the best results.
- another approach might be to use an algorithm approppriate for this type of data. For that, there is a package very similar to sklearn called [Imbalanced Learn](https://imbalanced-learn.org/stable/) and add the class imbalance to the class_weight parameter in most sklearn algorithms (including in [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)). 

You can read more about it [here](https://medium.com/eni-digitalks/imbalanced-data-an-extensive-guide-on-how-to-deal-with-imbalanced-classification-problems-6c8df0bc2cab).

### 3.5 Your model

Use the insigths learned from the models previously presented to build your own model with the framework you think is most suitable. You are free to use any of the code presented in the notebook.

###### ✏️ Implement your model

In [ ]:
# (Optional)

## 4. Model Evaluation

To be able to develop an ML model for the recalibration of SpO2 levels and implement it in a clinical setting, it is crucial to properly evaluate its performance in the task that is supposed to do.

A set of performance metrics should be carefully chosen, considering the clinical setting where the model will be aplied in but also the dataset where is what trained on:
- If the dataset contains mostly one racial group, how will it perform on others patients?
- Does the dataset have patients from a wide range of ages or is it more focused on a narrow range?

### 4.1 Performance Metrics

#### Regression Model

Regression tasks are often evaluated using the [$R^2$ score](https://en.wikipedia.org/wiki/Coefficient_of_determination). The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). In the general case when the true y is non-constant, a constant model that always predicts the average y disregarding the input features would get a $R^2$ score of 0.0.

In [ ]:
# Evaluate the linear model with R2 :
y_pred_train = lpipeline.predict(X_train_processed.values)
linear_train_r2 = r2_score(y_train_r, y_pred_train)
print(f'Linear Regression R2 score: {linear_train_r2}')

# Evaluate the ridge model with R2 :
y_pred_train = ridge_pipeline.predict(X_train_processed.values)
ridge_train_r2 = r2_score(y_train_r, y_pred_train)
print(f'Ridge Regression R2 score: {ridge_train_r2}')

✏️ Different regression metrics are [implemented by sklearn](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics). Try another metric that you think is relevant to this problem.

In [ ]:
# Code here !

...

We saw that the regression metrics of both the linear regression and the regularized linear ridge regressions are not very satisfying. There are two explanations: either, we have not the proper variables to explain the output variables or these variables do not relate to the SaO2 with a pure linear relationship.   

✏️  Can you fit and evaluate a better regression model in term of $R^2$ ? Consider for example [decision trees](https://scikit-learn.org/stable/modules/tree.html#regression) or [ensemble models](https://scikit-learn.org/stable/modules/ensemble.html). But feel free to experiment with any other type of regression algorithms ! 

In [ ]:
# (Optional)

In [ ]:
visualizer = ResidualsPlot(ridge_pipeline)
visualizer.fit(X_train_processed.values, y_train_r)
visualizer.score(X_test_processed.values, y_test_r)
visualizer.show()

In [ ]:
ridge_pipeline.fit(X_train_processed.values, y_train_r)
hat_y_test = ridge_pipeline.predict(X_test_processed.values)

fig, ax = plt.subplots(1)
sns.regplot(data=None, x=y_test_r, y=hat_y_test, line_kws={"color":"black", "linestyle":"dotted"})
plt.plot()
ax.set(xlabel=r"$y$", ylabel=r"$\hat y$")
test_r2_score_ = r2_score(y_test_r, hat_y_test)
print(f"R2 score: {test_r2_score_}")

#### Classification Model

In [ ]:
#Instantiate the classification model and visualizer
visualizer = ClassificationReport(SVC(), classes=[0,1], support=True)

visualizer.fit(X_train_processed.values, y_train_c.ravel())        # Fit the visualizer and the model
visualizer.score(X_test_processed.values, y_test_c.ravel())        # Evaluate the model on the test data
visualizer.show()       

In [ ]:
visualizer = ClassPredictionError(
    SVC(), classes=[0,1])
visualizer.fit(X_train_processed.values, y_train_c.ravel())
visualizer.score(X_test_processed.values, y_test_c.ravel())
visualizer.show()

#### ✏️ Properly evaluate your model

In [ ]:
# (Optional)

### 4.2 Feature Importance

Compute the SHAP(SHapley Additive exPlanations) values for the test data

In [ ]:
best_predictor = lpipeline.fit(X_train, y_train_r)
best_predictor

In [ ]:
%%time
explainer = shap.Explainer(best_predictor.predict, X_train)
shap_values = explainer(X_test)

Plot the SHAP values for each feature

In [ ]:
fig, ax = plt.subplots(figsize=(6, 8))
shap.plots.beeswarm(shap_values, max_display=10, show=False)
plt.title("Feature Importance: SHAP Values for Top 10 Features", fontsize=14)
plt.tight_layout()
plt.show()

Plot the most important features

In [ ]:
fig, ax = plt.subplots(figsize=(6, 8))
shap.plots.bar(shap_values, max_display=10, show=False)
plt.tight_layout()